In [37]:
%pip install --quiet pandas plotly networkx pyvis
%pip install -q --upgrade nbformat ipython
%pip install ipywidgets


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# %%

from ipywidgets import Dropdown
from IPython.display import display
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
from pyvis.network import Network
from IPython.display import IFrame

pd.options.display.max_rows = 20
FILE = "./assists_format.csv"          # adjust if the file lives elsewhere

: 

In [3]:
# %%
df = pd.read_csv(FILE)
df.head(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8072 entries, 0 to 8071
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   FASE           8072 non-null   object
 1   JORNADA        8072 non-null   int64 
 2   GAME           8072 non-null   object
 3   EQUIPO         8072 non-null   object
 4   PASADOR        8072 non-null   object
 5   ANOTADOR       8072 non-null   object
 6   N_ASISTENCIAS  8072 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 441.6+ KB


(                 FASE  JORNADA                                   GAME  \
 0  Liga Regular "B-A"        1  CB LA MATANZA vs MOVISTAR ESTUDIANTES   
 1  Liga Regular "B-A"        1  CB LA MATANZA vs MOVISTAR ESTUDIANTES   
 2  Liga Regular "B-A"        1  CB LA MATANZA vs MOVISTAR ESTUDIANTES   
 3  Liga Regular "B-A"        1  CB LA MATANZA vs MOVISTAR ESTUDIANTES   
 4  Liga Regular "B-A"        1  CB LA MATANZA vs MOVISTAR ESTUDIANTES   
 
                  EQUIPO                 PASADOR             ANOTADOR  \
 0         CB LA MATANZA     R. HERNÁNDEZ CHÁVEZ    A. HERNANDEZ DIAZ   
 1         CB LA MATANZA         F. PIRELA PEREZ    A. HERNANDEZ DIAZ   
 2         CB LA MATANZA           A. GOMEZ PEÑA      F. PIRELA PEREZ   
 3         CB LA MATANZA         F. PIRELA PEREZ  P. PERDOMO MARTÍNEZ   
 4  MOVISTAR ESTUDIANTES  S. RODRIGUEZ HERNANDEZ    V. NEGRE FABREGAT   
 
    N_ASISTENCIAS  
 0              1  
 1              2  
 2              1  
 3              1  
 4            

In [10]:
# %%
# Top 15 passer-scorer pairs overall
pairs = (df
         .groupby(['EQUIPO', 'PASADOR', 'ANOTADOR'], as_index=False)
         .agg(N_ASISTENCIAS=('N_ASISTENCIAS', 'sum'))
         .sort_values('N_ASISTENCIAS', ascending=False))
pairs


,EQUIPO,PASADOR,ANOTADOR,N_ASISTENCIAS
2698,UROS DE RIVAS,D. LORENZO NEGRETE,D. GARCIA CASARRUBIOS CASTRO,33
1275,FLEXICAR FUENLABRADA,P. RODRIGUEZ PASCUAL,D. SANCHEZ PEREZ,31
2525,TOSCONES CORRALEJO,G. GUERRA LOZANO,E. OGUNFOLU,30
369,C.B. TRES CANTOS,G. DIAZ MONTERO,J. DOMINGUEZ LARRE,29
2541,TOSCONES CORRALEJO,K. RODRIGUEZ AGUILERA,E. OGUNFOLU,28
...,...,...,...,...
2867,ZENTRO BASKET MADRID,M. HARO HARRIS,D. GARCIA DELGADO,1
31,BALONCESTO ALCALA,D. MARTIN LLUNA,V. DEL GROSSO,1
30,BALONCESTO ALCALA,D. MARTIN LLUNA,T. BOUNDI,1
28,BALONCESTO ALCALA,D. MARTIN LLUNA,M. ECHEVERRIA FERNANDEZ,1


In [11]:
# %%
fig = px.bar(pairs.head(15),
             x='N_ASISTENCIAS',
             y='PASADOR',
             color='EQUIPO',
             orientation='h',
             hover_data=['ANOTADOR'],
             title='Top passer → scorer synergies (all season)')
fig.update_layout(yaxis=dict(autorange='reversed'))
fig.show()


In [ ]:
import plotly.express as px
from ipywidgets import Dropdown, Output, VBox   # already installed with jupyter-lab/notebook
from IPython.display import display, clear_output

def make_heat(team: str):
    """Return a PASADOR x ANOTADOR matrix (0 when no assists)."""
    return (df[df["EQUIPO"] == team]
              .pivot_table(index="PASADOR",
                           columns="ANOTADOR",
                           values="N_ASISTENCIAS",
                           aggfunc="sum",
                           fill_value=0))

def build_fig(team: str):
    heat = make_heat(team)
    fig  = px.imshow(heat,
                     text_auto=True,
                     aspect="auto",
                     color_continuous_scale="Blues")
    fig.update_xaxes(side="top")
    return fig

# 3️⃣ widgets ──────────────────────────────────────────────────────────
teams          = sorted(df["EQUIPO"].unique())
team_dropdown  = Dropdown(options=teams, value=teams[0], description="Team:")
out            = Output()

def on_change(change):
    """Called when dropdown value changes."""
    if change["name"] == "value" and change["type"] == "change":
        with out:
            clear_output(wait=True)          # wipe previous plot
            build_fig(change["new"]).show()  # redraw

team_dropdown.observe(on_change, names="value")

# 4️⃣ first render & display ───────────────────────────────────────────
with out:
    build_fig(team_dropdown.value).show()

display(VBox([team_dropdown, out]))

In [ ]:
from pyvis.network import Network
import networkx as nx
import os, webbrowser
from IPython.display import HTML, display

# --- build your DiGraph with a `weight` attribute ---
G = nx.DiGraph()
for _, r in df.iterrows():
    G.add_edge(
        r['PASADOR'], 
        r['ANOTADOR'],
        weight = r['N_ASISTENCIAS'],
        title  = f"{r['PASADOR']} → {r['ANOTADOR']}: {r['N_ASISTENCIAS']}"
    )

# --- push it into PyVis ---
net = Network(
    height="650px", width="100%", 
    directed=True, notebook=False,
    bgcolor="#ffffff", font_color="black"
)
net.from_nx(G)

# --- now set your edge widths robustly ---
for e in net.edges:
    # first try the PyVis‐side value
    w = e.get("value")
    if w is None:
        # fallback to the NX graph
        w = G.get_edge_data(e["from"], e["to"], {}).get("weight", 1)
    e["width"] = 1 + w * 0.25

# --- save + display ---
html_file = "assist_network.html"
net.save_graph(html_file)

# A) force‐open in your browser:
webbrowser.open(f"file://{os.path.abspath(html_file)}", new=2)


In [24]:
# 7
ts = (df
      .groupby(['JORNADA'])
      .agg(TOTAL_ASISTS=('N_ASISTENCIAS', 'sum'))
      .reset_index())

fig = px.line(ts, x='JORNADA', y='TOTAL_ASISTS',
              markers=True,
              title='Total recorded assists per Jornada')
fig.show()


In [35]:
team_totals = df.groupby('EQUIPO', as_index=False)['N_ASISTENCIAS'].sum()
fig = px.bar(team_totals, x='EQUIPO', y='N_ASISTENCIAS', title='Total assists per Team')
fig.update_layout(xaxis_title='Team', yaxis_title='Total assists', xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
# Merge with results
results = pd.read_csv("resultados_completos.csv")   # columns as defined earlier
merged  = (df.merge(results,
                    left_on=['JORNADA', 'EQUIPO'],
                    right_on=['Jornada', 'Local']))

clutch = (merged[merged['Resultado'] == 'Gano']
          .groupby(['PASADOR', 'ANOTADOR'], as_index=False)
          .agg(Asists_in_Wins=('N_ASISTENCIAS', 'sum'))
          .sort_values('Asists_in_Wins', ascending=False)
          .head(10))

px.bar(clutch,
       x='Asists_in_Wins', y='PASADOR',
       color='ANOTADOR', orientation='h',
       title='Who feeds whom – in wins only').update_layout(yaxis={'categoryorder':'total ascending'}).show()
